In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.environ["DATA_ROOT"] = "/content/drive/MyDrive/hazard-data"

In [ ]:
import sys
sys.path.append("..")

from src.utils import load_config, get_output_dir, ensure_dir
from src.gee_pipeline import init_ee, get_s2_collection, load_boundary_fc, load_classes, make_class_mask
from src.tiling import generate_tiles
from src.export import get_first_tile_url

cfg = load_config("../configs/config.yaml")
out_dir = get_output_dir(cfg)
ensure_dir(out_dir)

In [ ]:
init_ee(auth=True)

In [ ]:
boundary_fc = load_boundary_fc(cfg["boundary_path"])
classes_list = load_classes(cfg)
aoi_geom = boundary_fc.geometry()

In [ ]:
coll = get_s2_collection(aoi_geom, cfg["start_date"], cfg["end_date"], cfg["cloud_max"])
mosaic = coll.select(cfg["bands"]).mosaic()

In [ ]:
label_mask = make_class_mask(classes_list, boundary_fc, scale=10)

In [ ]:
tiles_fc = generate_tiles(coll, aoi_geom, tile_size=cfg["tile_size"], bands=cfg["bands"], scale=10)
print("Tiles:", tiles_fc.size().getInfo())

In [ ]:
img_url = get_first_tile_url(mosaic, tiles_fc, bands=cfg["bands"], scale=10, fmt="PNG")
print("Image tile URL:\n", img_url)

from ee import Feature
first_tile_geom = Feature(tiles_fc.first()).geometry()
mask_url = label_mask.clip(first_tile_geom).getDownloadURL({"scale": 10, "region": first_tile_geom, "format": "GEO_TIFF"})
print("Mask tile URL:\n", mask_url)